# FineWeb-Edu 10B subset

https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu?row=5

In [101]:
from datasets import load_dataset  # pip install datasets

In [5]:
download_dir = "/mnt/md0/data"
fw = load_dataset("HuggingFaceFW/fineweb-edu", name="sample-10BT", split="train", cache_dir=download_dir)

Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9672101 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

In [6]:
fw

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_rows: 9672101
})

In [8]:
fw[1]

{'text': "Taking Play Seriously\nBy ROBIN MARANTZ HENIG\nPublished: February 17, 2008\nOn a drizzly Tuesday night in late January, 200 people came out to hear a psychiatrist talk rhapsodically about play -- not just the intense, joyous play of children, but play for all people, at all ages, at all times. (All species too; the lecture featured touching photos of a polar bear and a husky engaging playfully at a snowy outpost in northern Canada.) Stuart Brown, president of the National Institute for Play, was speaking at the New York Public Library's main branch on 42nd Street. He created the institute in 1996, after more than 20 years of psychiatric practice and research persuaded him of the dangerous long-term consequences of play deprivation. In a sold-out talk at the library, he and Krista Tippett, host of the public-radio program ''Speaking of Faith,'' discussed the biological and spiritual underpinnings of play. Brown called play part of the ''developmental sequencing of becoming a 

# Tokenize the dataset

In [10]:
from transformers import LlamaTokenizerFast
enc = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer") # 32000
enc

LlamaTokenizerFast(name_or_path='hf-internal-testing/llama-tokenizer', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [41]:
encoded = enc.encode(fw[1]["text"])
decoded = enc.decode(encoded)
len(encoded), len(decoded), decoded == fw[1]["text"]

(1206, 5259, False)

In [45]:
decoded[:100], enc.tokenize(decoded)[:5], enc.tokenize(fw[1]["text"])[:5]

('<s> Taking Play Seriously\nBy ROBIN MARANTZ HENIG\nPublished: February 17, 2008\nOn a drizzly Tuesday n',
 ['▁<s>', '▁T', 'aking', '▁Play', '▁Ser'],
 ['▁T', 'aking', '▁Play', '▁Ser', 'iously'])

In [16]:
fw[1]["text"][:100]

'Taking Play Seriously\nBy ROBIN MARANTZ HENIG\nPublished: February 17, 2008\nOn a drizzly Tuesday night'

In [25]:
# Search all occurances of enc.special_tokens_map["eos_token"] in decoded
import re
for k, v in enc.special_tokens_map.items():
    print(k, v, len(re.findall(v, decoded)))

bos_token <s> 1
eos_token </s> 0
unk_token <unk> 0


In [31]:
encoded = enc.encode(fw[1]["text"], add_special_tokens=False)
decoded = enc.decode(encoded)
decoded[:100], decoded == fw[1]["text"]

('Taking Play Seriously\nBy ROBIN MARANTZ HENIG\nPublished: February 17, 2008\nOn a drizzly Tuesday night',
 True)

In [77]:
import numpy as np

def tokenize(document):
    tokens = enc.encode(enc.special_tokens_map["bos_token"], add_special_tokens=False)
    tokens.extend(enc.encode(document["text"], add_special_tokens=False))
    tokens_np = np.asarray(tokens)
    assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), "token dictionary too large for uint16"
    # 2**16 = 65536 which is always true for llama tokenizer with 32000 vocab size
    tokens_np = tokens_np.astype(np.uint16)
    return tokens_np


tokenize(fw[1])

array([    1,   323,  5086, ...,  1708,  3109, 29889], dtype=uint16)

# Sharding

All documents become a massive array of uint16 tokens.
We can shard this array into smaller arrays and save them as numpy arrays.
We can also save the metadata in a separate file.

Now every shard will have roughly the same number of tokens.
They will not contain incomplete documents.
To shuffle the dataset, we can shuffle the order of the shards and then shuffle the documents within each shard.

In [106]:
from copy import deepcopy
from multiprocessing import Pool
from os import makedirs, cpu_count, environ
from tqdm import tqdm
import numpy as np
import json

# Configuration
environ["TOKENIZERS_PARALLELISM"] = "true"
max_tokens_per_shard = 100_000_000  # 100M tokens, 191mb, 84k docs
max_shards = np.inf
save_dir = "/mnt/ssd/data/fineweb-edu-10BT/llama-tokenizer"
makedirs(save_dir, exist_ok=True)

# Metadata Initialization
shard_metadata_template = {"source_indices": [], "bos_indices": [], "num_tokens": 0, "num_documents": 0}
complete_metadata = {
    "dataset_name": "fineweb-edu-10BT",
    "tokenizer_name": "hf-internal-testing/llama-tokenizer",
    "max_tokens_per_shard": max_tokens_per_shard,
    "num_training_tokens": 0,
    "train_shards": [],  # Contains indices of shards
    "validation_shards": [],
    "test_shards": [],
    "num_shards": 0,
    "num_tokens": [],  # contains number of tokens in each shard
    "num_documents": [],  # contains number of documents in each shard
}

# Helper function to save shard data
def save_shard_data(shard_idx, tokens, metadata):
    np.save(f"{save_dir}/tokens_shard_{shard_idx:03d}.npy", tokens)
    with open(f"{save_dir}/metadata_shard_{shard_idx:03d}.json", "w") as f:
        json.dump(metadata, f, indent=2)
    complete_metadata["num_tokens"].append(metadata["num_tokens"])
    complete_metadata["num_documents"].append(metadata["num_documents"])

# Main processing loop
current_document_index = 0
current_shard_index = 0
current_shard_tokens = 0
shard_tokens = np.zeros(max_tokens_per_shard, dtype=np.uint16)
shard_metadata = deepcopy(shard_metadata_template)

n_procs = max(1, int(cpu_count() * 0.5))  # 128 on MJ
with Pool(n_procs) as pool:
    # Without multiprocessing
    # for i, tokens_np in enumerate(tqdm(map(tokenize, fw), total=len(fw))):
    # 100%|██████████| 9672101/9672101 [41:01<00:00, 3929.23it/s] with 128 procs
    for i, tokens_np in enumerate(tqdm(pool.imap(tokenize, fw), total=len(fw))):
        if current_shard_tokens + len(tokens_np) > max_tokens_per_shard:
            save_shard_data(current_shard_index, shard_tokens[:current_shard_tokens], shard_metadata)
            current_shard_index += 1
            current_shard_tokens = 0
            shard_tokens.fill(0)
            shard_metadata = deepcopy(shard_metadata_template)
            complete_metadata["num_shards"] += 1
            if current_shard_index >= max_shards:
                break

        shard_tokens[current_shard_tokens:current_shard_tokens + len(tokens_np)] = tokens_np
        shard_metadata["bos_indices"].append(current_shard_tokens)
        current_shard_tokens += len(tokens_np)
        shard_metadata["source_indices"].append(i)
        shard_metadata["num_tokens"] += len(tokens_np)
        shard_metadata["num_documents"] += 1
        current_document_index += 1

    if current_shard_tokens > 0:
        save_shard_data(current_shard_index, shard_tokens[:current_shard_tokens], shard_metadata)

# Split the shards into train, validation, and test in 90:5:5 ratio
val_shard_start = int(complete_metadata["num_shards"] * 0.9)
test_shard_start = int(complete_metadata["num_shards"] * 0.95)
# Shuffle the shards
shard_indices = np.arange(complete_metadata["num_shards"])
np.random.shuffle(shard_indices)
complete_metadata["train_shards"] = shard_indices[:val_shard_start].tolist()
complete_metadata["validation_shards"] = shard_indices[val_shard_start:test_shard_start].tolist()
complete_metadata["test_shards"] = shard_indices[test_shard_start:].tolist()
complete_metadata["num_training_tokens"] = sum([complete_metadata["num_tokens"][i] for i in complete_metadata["train_shards"]])

# Save the complete metadata as json
with open(f"{save_dir}/metadata.json", "w") as f:
    json.dump(complete_metadata, f, indent=2)
    
complete_metadata

100%|██████████| 9672101/9672101 [38:20<00:00, 4203.53it/s]  


{'dataset_name': 'fineweb-edu-10BT',
 'tokenizer_name': 'hf-internal-testing/llama-tokenizer',
 'max_tokens_per_shard': 100000000,
 'num_training_tokens': 10199788391,
 'train_shards': [7,
  75,
  107,
  31,
  62,
  11,
  19,
  50,
  91,
  66,
  86,
  13,
  5,
  68,
  43,
  93,
  73,
  87,
  72,
  46,
  15,
  113,
  41,
  76,
  10,
  60,
  96,
  8,
  28,
  52,
  74,
  79,
  3,
  26,
  32,
  23,
  70,
  105,
  25,
  89,
  53,
  6,
  18,
  17,
  100,
  104,
  4,
  21,
  34,
  51,
  22,
  35,
  12,
  82,
  78,
  57,
  88,
  58,
  98,
  45,
  0,
  24,
  65,
  47,
  36,
  14,
  109,
  111,
  103,
  30,
  54,
  85,
  95,
  2,
  59,
  49,
  42,
  94,
  16,
  29,
  61,
  38,
  56,
  80,
  92,
  112,
  97,
  48,
  81,
  90,
  33,
  63,
  39,
  69,
  83,
  44,
  20,
  37,
  64,
  108,
  9,
  40],
 'validation_shards': [106, 110, 1, 101, 102, 71],
 'test_shards': [27, 99, 55, 84, 67, 77],
 'num_shards': 114,
 'num_tokens': [99999023,
  99989972,
  99999006,
  99997050,
  99998535,
  99999923,
  9

In [ ]:
! ls /mnt/ssd/data/fineweb-edu-10BT/llama-tokenizer